<a href="https://colab.research.google.com/github/safikhanSoofiyani/CS6910-Assignment-2/blob/main/CS6910_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import numpy as np
import tensorflow as tf
import random

# Importing libraries related to Image Preprocessing
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

# Importing libraries related to CNN Model building
from tensorflow import keras
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense,Flatten,Conv2D,MaxPool2D,Activation,Dropout,BatchNormalization

random.seed(hash("seriously you compete with me") % 2**32 - 1)
np.random.seed(hash("i am mohammed safi") % 2**32 - 1)
tf.random.set_seed(hash("ur rahman khan") % 2**32 - 1)


In [2]:
from tensorflow.python.keras import backend as K
K._get_available_gpus()

['/device:GPU:0']

In [3]:
!nvidia-smi

Sat Mar 26 17:04:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    32W / 250W |    375MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!pip install wandb
import wandb

from wandb.keras import WandbCallback

     |████████████████████████████████| 1.7 MB 5.1 MB/s 
     |████████████████████████████████| 144 kB 62.9 MB/s 
     |████████████████████████████████| 181 kB 73.4 MB/s 
     |████████████████████████████████| 63 kB 2.3 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=b6b67fe7cc3f787a42d99a0a46a59aa1df8c8848d1c24ab89f9ba96459021583
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# # Creating a CNN: number of filters, size of filters, activation function has to be 
# # user specified
# img_shape=(256,256,3)
# def build_cnn( batch_norm, num_filters, dropout, conv_activation, dense_activation, filter_size, pool_kernel_size, dense_layer):
#   # batch_norm       : "Boolean" set to True, if you are using batch normalization
#   # num_filters      : "List" number of activation filters for each layer
#   # dropout          : "float or double" specify the dropout % for regularization (in decimals)
#   # conv_activation  : "List" activation used for convolution layer
#   # dense_activation : "String" acitvation used for densely connected layers
#   # filter_size      : "List" kernel sizes for convultion layers
#   # pool_kernel_size : "List" kernel sizes for maxpooling layers
#   # dense_layer      : "Integer" number of neurons in the dense layer
  
  
#   model=Sequential()
#   # Adding the first conv,activ,maxpool layers
#   model.add(Conv2D(filters = num_filters[0],
#                    kernel_size = filter_size[0],
#                    input_shape=img_shape))
  
#   # Add batch normalization layer, if the user specifies
#   if batch_norm:
#     model.add(BatchNormalization())
  
#   model.add(Activation(conv_activation[0]))

#   model.add(MaxPool2D(pool_size = pool_kernel_size[0], strides = (2,2)))

#   # Adding the next 4 layer blocks
#   for i in range(1,5):
#     model.add(Conv2D(filters = num_filters[i],kernel_size = filter_size[i]))
#     if batch_norm:
#       model.add(BatchNormalization())
#     model.add(Activation(conv_activation[i]))
#     model.add(MaxPool2D(pool_size = pool_kernel_size[i], strides = (2,2)))
  
#   # Flattening the feature map to a column vector
#   model.add(Flatten())
#   model.add(Dense(units = dense_layer, activation = dense_activation))
#   # Adding dropout regularization
#   model.add(Dropout(dropout))
#   # Adding the final Dense layer with dimensions equal to number of classes and softmax activation
#   model.add(Dense(10, activation = "softmax"))

#   return model


In [18]:
# Creating a CNN: number of filters, size of filters, activation function has to be 
# user specified
img_shape=(256,256,3)
def build_cnn(conv_activation , dense_activation, num_filters, conv_filter_size, pool_filter_size, batch_norm, dense_layer, dropout):
  # conv_activation         : "List" activation used for convolution layer
  # dense_act        : "String" acitvation used for densely connected layers
  # num_filters      : "List" number of activation filters for each layer
  # conv_filter_size : "List" kernel sizes for convultion layers
  # pool_filter_size : "List" kernel sizes for maxpooling layers
  # batch_norm       : "Boolean" set to True, if you are using batch normalization
  # dim_final        : "Integer" dimensionality of output space after 5 blocks of convultion, maxpooling blocks
  # dropout          : "float or double" specify the dropout % for regularization (in decimals)
  
  model=Sequential()
  # Adding the first conv,activ,maxpool layers
  model.add(Conv2D(filters=num_filters[0],
                   kernel_size=conv_filter_size[0],
                   input_shape=img_shape))
  
  # Add batch normalization layer, if the user specifies
  if batch_norm:
    model.add(BatchNormalization())
  
  model.add(Activation(conv_activation[0]))
  model.add(MaxPool2D(pool_size=pool_filter_size[0],strides=(2,2)))

  # Adding the next 4 layer blocks
  for i in range(1,5):
    model.add(Conv2D(filters=num_filters[i],kernel_size=conv_filter_size[i]))
    if batch_norm:
      model.add(BatchNormalization())
    model.add(Activation(conv_activation[i]))
    model.add(MaxPool2D(pool_size=pool_filter_size[i],strides=(2,2)))
  
  # Flattening the feature map to a column vector
  model.add(Flatten())
  model.add(Dense(units=dense_layer ,activation = dense_activation))
  # Adding dropout regularization
  model.add(Dropout(dropout))
  # Adding the final Dense layer with dimensions equal to number of classes and softmax activation
  model.add(Dense(10,activation="softmax"))

  return model

In [7]:
conv=["relu","relu","relu","relu","relu"]
den="relu"
filt=[32,64,128,256,512]
conv_size=[(3, 3), (3, 3), (3, 3), (5, 5), (7, 7)]
pool_size = [(2, 2), (2, 2), (2, 2), (2, 2), (2, 2)]
model=build_cnn(conv,den,filt,conv_size,pool_size,True,512,0.8)

Input pipeline for image classification reference :
https://towardsdatascience.com/what-is-the-best-input-pipeline-to-train-image-classification-models-with-tf-keras-eb3fe26d3cc5

In [12]:
# Dataset without augmentation

# Enter the path of dataset
path=r"/content/drive/MyDrive/nature_12K/inaturalist_12K/train"

# Training dataset:
train_dataset=tf.keras.preprocessing.image_dataset_from_directory(
    directory=path,
    labels='inferred',
    label_mode='categorical',
    color_mode='rgb',
    batch_size=32,
    image_size=(256,256),
    shuffle=True,
    seed=19,
    validation_split=0.1,
    subset='training'
)
# Validation dataset:
valid_dataset=tf.keras.preprocessing.image_dataset_from_directory(
    directory=path,
    labels='inferred',
    label_mode='categorical',
    color_mode='rgb',
    batch_size=32,
    image_size=(256,256),
    shuffle=True,
    seed=19,
    validation_split=0.1,
    subset='validation'
)

Found 9999 files belonging to 10 classes.
Using 9000 files for training.
Found 9999 files belonging to 10 classes.
Using 999 files for validation.


In [13]:
len_train,len_valid=len(train_dataset),len(valid_dataset)
# # Retaining 10% of data for hyperparameter tuning
# train_data=train_dataset.take(int(0.9*len_train))
# valid_data=valid_dataset.take(int(0.1*len_valid))
print(len(train_dataset))

282


In [17]:
# Dataset with augmentation
path=r"/content/drive/MyDrive/nature_12K/inaturalist_12K/train"
training_data_augmentation=ImageDataGenerator(rescale=1./255,
                                        height_shift_range=0.2,
                                        width_shift_range=0.2,
                                        horizontal_flip=True,
                                        zoom_range=0.2,
                                        fill_mode="nearest",
                                        validation_split = 0.1)

# Validation data is not being augmented
validation_data_augmentation=ImageDataGenerator(
    validation_split=0.1
)

train_aug=training_data_augmentation.flow_from_directory(path,shuffle=True,seed=19,subset='training')
valid_aug=validation_data_augmentation.flow_from_directory(path,shuffle=True,seed=19,subset='validation')



Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.


In [18]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss=tf.keras.losses.CategoricalCrossentropy(name='loss'),
              metrics=[tf.keras.metrics.CategoricalAccuracy(name='acc')])

history=model.fit(x=train_aug,
                  epochs=10,
                  validation_data=valid_aug,callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_acc', patience = 5)]
                  )

Epoch 1/10
282/282 [==============================] - 240s 850ms/step - loss: 2.1631 - acc: 0.2042 - val_loss: 6.4374 - val_acc: 0.1311
Epoch 2/10
282/282 [==============================] - 234s 829ms/step - loss: 2.1433 - acc: 0.2123 - val_loss: 5.8009 - val_acc: 0.1512
Epoch 3/10
282/282 [==============================] - 233s 825ms/step - loss: 2.1249 - acc: 0.2214 - val_loss: 26.9332 - val_acc: 0.1481
Epoch 4/10
282/282 [==============================] - 232s 824ms/step - loss: 2.1291 - acc: 0.2188 - val_loss: 109.4804 - val_acc: 0.1301
Epoch 5/10
282/282 [==============================] - 245s 869ms/step - loss: 2.0949 - acc: 0.2309 - val_loss: 106.5589 - val_acc: 0.1732
Epoch 6/10
282/282 [==============================] - 241s 856ms/step - loss: 2.0807 - acc: 0.2476 - val_loss: 82.7952 - val_acc: 0.1411
Epoch 7/10
282/282 [==============================] - 234s 828ms/step - loss: 2.0762 - acc: 0.2507 - val_loss: 151.6951 - val_acc: 0.1692
Epoch 8/10
282/282 [====================

In [19]:
def get_data():
    path=r"/content/drive/MyDrive/nature_12K/inaturalist_12K/train"

    # Training dataset:
    train_dataset=tf.keras.preprocessing.image_dataset_from_directory(
        directory=path,
        labels='inferred',
        label_mode='categorical',
        color_mode='rgb',
        batch_size=32,
        image_size=(256,256),
        shuffle=True,
        seed=19,
        validation_split=0.1,
        subset='training'
    )
    # Validation dataset:
    valid_dataset=tf.keras.preprocessing.image_dataset_from_directory(
        directory=path,
        labels='inferred',
        label_mode='categorical',
        color_mode='rgb',
        batch_size=32,
        image_size=(256,256),
        shuffle=True,
        seed=19,
        validation_split=0.1,
        subset='validation'
    )
    
    #return train_data, valid_data
    return train_dataset, valid_dataset


In [20]:
def get_augmented_data():
    training_data_augmentation=ImageDataGenerator(rescale=1./255,
                                        height_shift_range=0.2,
                                        width_shift_range=0.2,
                                        horizontal_flip=True,
                                        zoom_range=0.2,
                                        fill_mode="nearest",
                                        validation_split = 0.1)

    # Validation data is not being augmented
    validation_data_augmentation=ImageDataGenerator(
        validation_split=0.1
    )

    train_aug=training_data_augmentation.flow_from_directory(path,shuffle=True,seed=19,subset='training')
    valid_aug=validation_data_augmentation.flow_from_directory(path,shuffle=True,seed=19,subset='validation')

    return train_aug, valid_aug


In [21]:
def train():
    config_defaults = {
      'batch_norm': True,
      'num_filters': 32,
      'filter_org': 0.5,
      'dropout': 0.0,
      'data_augmentation': True,
      'num_epochs' : 10,
      'batch_size': 64,
      'dense_layer': 64,
      'learning_rate': 0.001,
      'kernel_size': 3
      
    }

  # Initializing the wandb run
    wandb.init(config=config_defaults)
    config = wandb.config

    conv_activation = ["relu","relu","relu","relu","relu"]
    dense_activation = "relu"

    num_filters = []
    filters = config.num_filters
    for i in range(5):
        num_filters.append(filters)
        filters = filters * config.filter_org
    
    conv_filter_size = []
    F = config.kernel_size
    for i in range(5):
        conv_filter_size.append((F,F))

    pool_filter_size = [(2, 2), (2, 2), (2, 2), (2, 2), (2, 2)]
    
   
    #tf.keras.backend.clear_session()

    #Creating model architecture here
    model = build_cnn(conv_activation, dense_activation, num_filters, conv_filter_size, 
                      pool_filter_size, config.batch_norm, config.dense_layer, 
                      config.dropout)
    model.summary()

    #Getting the data here


    if config.data_augmentation:
        train_data, valid_data = get_augmented_data()
    else:
        train_data, valid_data = get_data()

    name_run = str(config.batch_norm) + "_" + str(config.num_filters) + \
                "_" + str(config.filter_org) + "_" + str(config.dropout) + \
                "_" + str(config.data_augmentation) + "_" + str(config.num_epochs)
    
    wandb.run.name = name_run
    wandb_log = True

    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss = tf.keras.losses.CategoricalCrossentropy(name='loss'),
              metrics = [tf.keras.metrics.CategoricalAccuracy(name='acc')])

    history = model.fit(x = train_data,
                  epochs=10,
                  validation_data=valid_data,
                  callbacks = [WandbCallback()]
                  )

    #wandb.run.save()
    wandb.run.finish()







In [26]:
entity_name = "safi-vamsi-cs6910"
project_name = "Assignment 2"

In [24]:
def sweeper(entity_name, project_name):
    #Declaring the necessary hyperparameters.
    hyperparameters={
    
    'batch_norm': {
        'values':[True, False]
    },
    'num_filter':{
        'values':[32, 64, 128, 256]

    },

    'filter_org':{
        'values':[0.5, 1, 2]
    },

    'dropout':{
        'values':[0.0, 0.5, 0.6, 0.4]
    },

    'data_augmentation':{
        'values':[True, False]
    },

    'num_epochs': {
        'values':[10, 20, 30]
    },

    'batch_size': {
        'values':[32, 64, 128]
    },

    'dense_layer': {
        'values':[32, 64, 128, 512]
    },

    'learning_rate': {
        'values':[0.001, 0.0001]
    },

    'kernel_size': {
        'values': [3, 5, 7]
    }
    
    }   

    #Using bayes method for hyperparameter sweeps to curb the unnecessary configurations
    sweep_config = {
      'method' : 'bayes',
      'metric' :{
          'name': 'val_acc',
          'goal': 'maximize'
      },
      'parameters': hyperparameters
    }

    sweep_id = wandb.sweep(sweep_config, entity=entity_name, project=project_name)
    wandb.agent(sweep_id, train)



In [25]:
sweeper(entity_name, project_name)

Create sweep with ID: b6ej9uqk
Sweep URL: https://wandb.ai/safi-vamsi-cs6910/assgn2%20trial/sweeps/b6ej9uqk


wandb: Agent Starting Run: bejuh12g with config:
wandb: 	batch_norm: True
wandb: 	batch_size: 32
wandb: 	data_augmentation: False
wandb: 	dense_layer: 64
wandb: 	dropout: 0.5
wandb: 	filter_org: 2
wandb: 	kernel_size: 3
wandb: 	learning_rate: 0.0001
wandb: 	num_epochs: 10
wandb: 	num_filter: 32


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 254, 254, 32)     128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 254, 254, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 batch_normalization_1 (Batc  (None, 125, 125, 64)     2

acc,▁▃▅▅▆▆▇███
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▆▅▅▄▃▃▂▁▁
val_acc,▁▆▅▆▆▇▆▇█▄
val_loss,█▅▅▄▅▄▂▂▁▆
acc,0.15489
best_epoch,8
best_val_loss,2.21225
epoch,9
loss,2.20076
val_acc,0.15816


wandb: Agent Starting Run: e5s7o22t with config:
wandb: 	batch_norm: False
wandb: 	batch_size: 64
wandb: 	data_augmentation: False
wandb: 	dense_layer: 64
wandb: 	dropout: 0.5
wandb: 	filter_org: 2
wandb: 	kernel_size: 7
wandb: 	learning_rate: 0.0001
wandb: 	num_epochs: 30
wandb: 	num_filter: 64


...

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 254, 254, 32)     128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 254, 254, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 batch_normalization_1 (Batc  (None, 125, 125, 64)     2

acc,▁▂▂▃▄▅▅▆▇█
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▇▆▆▅▄▄▃▂▁
val_acc,▂▁▄▃▄▆▇█▇█
val_loss,█▆▅▅▅▃▃▂▂▁
acc,0.36944
best_epoch,9
best_val_loss,1.89083
epoch,9
loss,1.71381
val_acc,0.33834


wandb: Agent Starting Run: w69fy4yr with config:
wandb: 	batch_norm: True
wandb: 	batch_size: 128
wandb: 	data_augmentation: False
wandb: 	dense_layer: 128
wandb: 	dropout: 0.4
wandb: 	filter_org: 2
wandb: 	kernel_size: 5
wandb: 	learning_rate: 0.0001
wandb: 	num_epochs: 10
wandb: 	num_filter: 256


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 252, 252, 32)      2432      
                                                                 
 batch_normalization (BatchN  (None, 252, 252, 32)     128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 252, 252, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 126, 126, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 122, 122, 64)      51264     
                                                                 
 batch_normalization_1 (Batc  (None, 122, 122, 64)     2

wandb: Ctrl + C detected. Stopping sweep.
